In [1]:
import pandas as pd
import numpy as np
import torch
import os
import sys
from tqdm import tqdm, trange

sys.path.append("../../")
import biked_commons
from biked_commons.design_evaluation.design_evaluation import *
from biked_commons.data_loading import data_loading
from biked_commons.conditioning import conditioning
from biked_commons.design_evaluation.scoring import *
from biked_commons.benchmark_models import generative_modeling_utils, benchmarking_utils


c:\Users\Lyler\mambaforge\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import importlib
importlib.reload(generative_modeling_utils)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = data_loading.load_bike_bench_train()

data_tens = torch.tensor(data.values, dtype=torch.float32, device=device)

scaler = generative_modeling_utils.TorchScaler(data_tens)

scaled_data_tens = scaler.scale(data_tens)

continuous_conditioning = generative_modeling_utils.sample_continuous(len(scaled_data_tens))


In [3]:
save_scores = False

In [4]:
importlib.reload(generative_modeling_utils)


def predict(cond_test, generate_fn, batch_size, D, G, noise_dim):
    cond_test = cond_test.to(device)
    predictions = []
    #use batch size
    for i in range(0, len(cond_test), batch_size):
        batch = cond_test[i:i + batch_size]
        pred = generate_fn(D, G, batch, noise_dim, device=device)
        pred = scaler.unscale(pred)
        predictions.append(pred)
    predictions = torch.concat(predictions, axis=0)
    return predictions



def train_and_score(model="VAE", cond_idx=None, save=save_scores):

    div_weight_dict = {
        "VAE": 1.0,
        "GAN": 1.0,
        "DDPM_guided": 10000,
        "DDPM_conditional": 1.0,
    }

    div_weight = div_weight_dict[model]
    aux_fn = generative_modeling_utils.get_diversity_loss_fn(scaler, data.columns, 
                                                         diversity_weight = div_weight, 
                                                         score_weight = 1.0, 
                                                         constraint_vs_objective_weight = 10, 
                                                         constraint_falloff = 10, 
                                                         dpp_batch = 16, 
                                                         device = device)

    noise_dim = 10
    batch_size = 512
    epochs = 1 if model != "DDPM_guided" else 1 # For Testing Purposes
    # epochs = 100 if model != "DDPM_guided" else 1000 # Guided DDPM is fast to train, slower in inference
    train_params = (batch_size, 0.002, 0.002, noise_dim, epochs, 2, 256)
    D, G, generate_fn = generative_modeling_utils.train_model(scaled_data_tens, model, train_params, aux_fn, cond_idx, device)

    if cond_idx is None:
        continuous_condition = generative_modeling_utils.sample_continuous(10000, split="test", randomize=False)
        predictions = predict(continuous_condition, generate_fn, batch_size, D, G, noise_dim).detach()
        main_scores, detailed_scores = benchmarking_utils.evaluate_cond(predictions, f"{model}-PA", data.columns, device=device, save=save)
    else:
        continuous_condition = generative_modeling_utils.sample_continuous(10, split="test", randomize=False).to(device)[cond_idx]
        continuous_condition = continuous_condition.repeat(1000, 1)
        predictions = predict(continuous_condition, generate_fn, batch_size, D, G, noise_dim).detach()
        main_scores, detailed_scores = benchmarking_utils.evaluate_uncond(predictions, f"{model}-PA", cond_idx, data.columns, device=device, save=save)
        
        
    return main_scores, detailed_scores, predictions


In [5]:
#conditional testing
for model in ["VAE", "GAN", "DDPM_guided", "DDPM_conditional"]:
    main_scores, detailed_scores, predictions = train_and_score(model = model, cond_idx=None, save = save_scores)

print("Main scores: ", main_scores)
print("Detailed scores: ", detailed_scores)

c:\Users\Lyler\Documents\Bike-Bench-Temporary\src\biked_commons\benchmark_models\../..\biked_commons\benchmark_models\generative_modeling_utils.py:570: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data).float()
100%|██████████| 7/7 [01:46<00:00, 15.26s/it, loss=0.9774]
39it [00:00, 384.16it/s]c:\Users\Lyler\Documents\Bike-Bench-Temporary\src\biked_commons\benchmark_models\../..\biked_commons\benchmark_models\generative_modeling_utils.py:143: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.

Main scores:  Hypervolume                     0.353880
Constraint Satisfaction Rate    0.534000
Maximum Mean Discrepancy        0.014964
dtype: float64
Detailed scores:  Min Objective Score: Usability Score - 0 to 1                                                                0.193993
Min Objective Score: Drag Force                                                                             10.967923
Min Objective Score: Knee Angle Error                                                                        0.000000
Min Objective Score: Hip Angle Error                                                                        31.643547
Min Objective Score: Arm Angle Error                                                                         0.000000
Min Objective Score: Cosine Distance to Embedding                                                            0.000059
Min Objective Score: Mass                                                                                    2.844460
Min 

In [6]:
#unconditional testing
for i in range(1): #replace with range(10) for full testing
    for model in ["VAE", "GAN", "DDPM_guided", "DDPM_conditional"]:
        main_scores, detailed_scores, predictions = train_and_score(model = model, cond_idx=i, save = save_scores)

        print("Main scores: ", main_scores)
        print("Detailed scores: ", detailed_scores)

c:\Users\Lyler\Documents\Bike-Bench-Temporary\src\biked_commons\benchmark_models\../..\biked_commons\benchmark_models\generative_modeling_utils.py:570: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data).float()
100%|██████████| 7/7 [00:10<00:00,  1.43s/it, L_KL=0.2245, L_R=0.9982, L_tot=10.4962, L_aux=9.4531, CSR=0.9727, MCS=1.1517]  


Main scores:  Hypervolume                     0.072991
Constraint Satisfaction Rate    0.912000
Maximum Mean Discrepancy        0.093408
dtype: float64
Detailed scores:  Min Objective Score: Usability Score - 0 to 1                                                                0.301619
Min Objective Score: Drag Force                                                                             23.583591
Min Objective Score: Knee Angle Error                                                                       25.399277
Min Objective Score: Hip Angle Error                                                                        49.029171
Min Objective Score: Arm Angle Error                                                                        86.787094
Min Objective Score: Cosine Distance to Embedding                                                            0.003588
Min Objective Score: Mass                                                                                    6.899267
Min 

c:\Users\Lyler\Documents\Bike-Bench-Temporary\src\biked_commons\benchmark_models\../..\biked_commons\benchmark_models\generative_modeling_utils.py:570: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data).float()
100%|██████████| 7/7 [00:09<00:00,  1.41s/it, L_D_real=0.9370, L_D_fake=0.2135, L_G=0.7026, L_aux=9.7336, CSR=0.8516, MCS=1.6651] 


Main scores:  Hypervolume                     0.000000
Constraint Satisfaction Rate    0.000000
Maximum Mean Discrepancy        0.092927
dtype: float64
Detailed scores:  Min Objective Score: Usability Score - 0 to 1                                                                 1.000000
Min Objective Score: Drag Force                                                                              29.636734
Min Objective Score: Knee Angle Error                                                                       197.293720
Min Objective Score: Hip Angle Error                                                                        807.759030
Min Objective Score: Arm Angle Error                                                                        843.490000
Min Objective Score: Cosine Distance to Embedding                                                             0.054179
Min Objective Score: Mass                                                                                    36.1922

c:\Users\Lyler\Documents\Bike-Bench-Temporary\src\biked_commons\benchmark_models\../..\biked_commons\benchmark_models\generative_modeling_utils.py:570: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data).float()
100%|██████████| 7/7 [00:02<00:00,  2.52it/s, loss=0.9701]
100it [00:53,  1.85it/s]
100it [00:53,  1.88it/s]


Main scores:  Hypervolume                     0.275960
Constraint Satisfaction Rate    0.664000
Maximum Mean Discrepancy        0.023797
dtype: float64
Detailed scores:  Min Objective Score: Usability Score - 0 to 1                                                                0.194069
Min Objective Score: Drag Force                                                                             13.197374
Min Objective Score: Knee Angle Error                                                                        0.000000
Min Objective Score: Hip Angle Error                                                                        30.685074
Min Objective Score: Arm Angle Error                                                                         2.350227
Min Objective Score: Cosine Distance to Embedding                                                            0.000211
Min Objective Score: Mass                                                                                    3.303786
Min 

c:\Users\Lyler\Documents\Bike-Bench-Temporary\src\biked_commons\benchmark_models\../..\biked_commons\benchmark_models\generative_modeling_utils.py:570: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data).float()
100%|██████████| 7/7 [00:09<00:00,  1.32s/it, loss=1.6687, L_aux=-0.0778, CSR=0.0526, MCS=31.9872]


Main scores:  Hypervolume                     0.234785
Constraint Satisfaction Rate    0.637000
Maximum Mean Discrepancy        0.123594
dtype: float64
Detailed scores:  Min Objective Score: Usability Score - 0 to 1                                                                0.215539
Min Objective Score: Drag Force                                                                             12.603612
Min Objective Score: Knee Angle Error                                                                        0.000000
Min Objective Score: Hip Angle Error                                                                        28.648510
Min Objective Score: Arm Angle Error                                                                         0.000000
Min Objective Score: Cosine Distance to Embedding                                                            0.002511
Min Objective Score: Mass                                                                                    4.183015
Min 